In [1]:
import os
from pathlib import Path
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
def load_data(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [3]:
data_root = r"C:/Users/hayashi/doutorado/mac6958--dados-redes/webrtc"

In [4]:
file_path = Path(data_root, 'data/processed/facial-1007-sequential.pkl')

In [5]:
data = load_data(file_path)

In [6]:
X,Y = data

In [7]:
X.shape, Y.shape

((34880, 30, 323), (34880, 5))

In [8]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Please install GPU version of TF


In [10]:
# pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Using cached sklearn-0.0.post11.tar.gz (3.6 kB)
  Using cached sklearn-0.0.post10.tar.gz (3.6 kB)
  Using cached sklearn-0.0.post9.tar.gz (3.6 kB)
  Using cached sklearn-0.0.post7.tar.gz (3.6 kB)
  Using cached sklearn-0.0.post5.tar.gz (3.7 kB)
  Using cached sklearn-0.0.post4.tar.gz (3.6 kB)
  Using cached sklearn-0.0.post1.tar.gz (3.6 kB)
  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-1.4.2-cp310-cp310-win_amd64.whl (10.6 MB)
  Using cached scipy-1.13.0-cp310-cp310-win_amd64.whl (46.2 MB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\hayashi\doutorado\mac6958--dados-redes\webrtc\venv.2\Scripts\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\hayashi\\AppData\\Local\\Temp\\pip-install-2feufshs\\sklearn_44a2f84c821b43ffbd09df268ed160f5\\setup.py'"'"'; __file__='"'"'C:\\Users\\hayashi\\AppData\\Local\\Temp\\pip-install-2feufshs\\sklearn_44a2f84c821b43ffbd09df268ed160f5\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\hayashi\AppData\Local\Temp\pip-pip-egg-info-jb7sql98'
         cwd: C:\Users\hayashi\AppData\Local\Temp\pip-install-2feufshs\sklearn_44a2f84c821b43ffbd09df268ed160f5\
    Complete output (15 lines):
    The 'sklearn' PyPI package is deprecated

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import MultiHeadAttention, Embedding

In [12]:
# Split the data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [13]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [14]:
from tensorflow.keras.layers import GlobalAveragePooling1D

In [15]:
# Define a Transformer block as a layer
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Build the model using the Functional API
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
x = TransformerBlock(embed_dim=323, num_heads=8, ff_dim=512)(inputs)
x = GlobalAveragePooling1D()(x)  # Reduce the sequence dimension
outputs = Dense(5)(x)  # Linear activation for regression
model = Model(inputs=inputs, outputs=outputs)

In [16]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mse'])

In [17]:
# Summary of the model
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 30, 323)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block (TransformerBlock) │ (None, 30, 323)             │       3,679,482 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 323)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │           1,620 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,681,102 (14.04 MB)

 Trainable params: 3,681,102 (14.04 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(X_train, Y_train, epochs=8, batch_size=64, validation_data=(X_val, Y_val))

Epoch 1/8
436/436 ━━━━━━━━━━━━━━━━━━━━ 100s 225ms/step - loss: 0.6305 - mse: 0.6305 - val_loss: 0.0016 - val_mse: 0.0016
Epoch 2/8
436/436 ━━━━━━━━━━━━━━━━━━━━ 96s 221ms/step - loss: 0.0047 - mse: 0.0047 - val_loss: 0.0022 - val_mse: 0.0022
Epoch 3/8
436/436 ━━━━━━━━━━━━━━━━━━━━ 96s 221ms/step - loss: 0.0041 - mse: 0.0041 - val_loss: 0.0030 - val_mse: 0.0030
Epoch 4/8
322/436 ━━━━━━━━━━━━━━━━━━━━ 23s 210ms/step - loss: 0.0036 - mse: 0.0036